# Deep Learning Lab using PyTorch and Common Helper Packages

Courtesy of ISLP

# Imports

General Imports

In [1]:
import numpy as np , pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
(LinearRegression ,
LogisticRegression ,
Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
(train_test_split ,
GridSearchCV)

Torch specific imports

In [6]:
# General torch imports
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

# torchmetrics, for further performance metrics
from torchmetrics import (MeanAbsoluteError, R2Score)
from torchinfo import summary
from torchvision.io import read_image

# pytorch lightning, for interface used for training
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

# for reproducibility,
# from pytorch_lightning.utilities.seed import seed_everything
# seed_everything (0, workers=True)
torch.use_deterministic_algorithms(True , warn_only=True)

Load data

In [7]:
from torchvision.datasets import MNIST , CIFAR100
from torchvision.models import (resnet50 ,
    ResNet50_Weights)
from torchvision.transforms import (Resize ,
    Normalize ,
    CenterCrop ,
    ToTensor)

ISLP specific imports

In [8]:
from ISLP.torch import (SimpleDataModule ,
SimpleModule ,
ErrorTracker ,
rec_num_workers)

from ISLP.torch.imdb import (load_lookup ,
load_tensor ,
load_sparse ,
load_sequential)

Misc imports

In [9]:
from glob import glob
import json

# Feed Forward NN's

## Single Layer NN

In [19]:
# load data
Hitters = load_data('Hitters').dropna()
n = Hitters.shape[0]

In [20]:
# use MS from ISLP to create a design matrix, then convert to a numpy array
model = MS(Hitters.columns.drop('Salary'), intercept=False)
X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary'].to_numpy()

# split into training and test
(X_train, X_test, Y_train, Y_test) = train_test_split(
    X, Y, test_size=1/3, random_state=1
)

Fit a linear model as a very quick baseline

In [21]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat = hit_lm.predict(X_test)
np.abs(Yhat - Y_test).mean()

259.71528833146243

Fit a lasso regression as a slightly less quick baseline

In [24]:
# scale data for the regularisation, then apply the lasso regularisation
# the Pipeline just combines these 2 steps in one for us
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True, max_iter=30000)
standard_lasso = Pipeline(steps=[
    ('scaler', scaler),
    ('lasso', lasso)
])

# set up a grid of lambda values for lasso
X_scaled = scaler.fit_transform(X_train)
n = X_scaled.shape[0]
lam_max = np.fabs(X_scaled.T.dot(Y_train - Y_train.mean()).max()) / n
param_grid = {'alpha': np.exp(np.linspace(0, np.log(0.01), 100)) * lam_max}

In [25]:
cv = KFold(10, shuffle=True, random_state=1)
grid = GridSearchCV(lasso, param_grid=param_grid, cv=cv, scoring='neg_mean_absolute_error')
grid.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=1, shuffle=True),
             estimator=Lasso(max_iter=30000, warm_start=True),
             param_grid={'alpha': array([255.65755026, 244.03752004, 232.94563812, 222.35789935,
       212.25138966, 202.60423642, 193.39556119, 184.60543446,
       176.21483255, 168.20559645, 160.5603925 , 153.26267486,
       146.29664975, 139.64724123, 133.3000586 , 127.24136521,
       121.45...
        10.81237873,  10.32093943,   9.8518368 ,   9.40405561,
         8.97662677,   8.56862523,   8.17916799,   7.80741218,
         7.45255325,   7.1138232 ,   6.79048896,   6.48185076,
         6.18724064,   5.906021  ,   5.63758323,   5.38134637,
         5.13675587,   4.90328239,   4.68042064,   4.4676883 ,
         4.26462497,   4.07079118,   3.88576744,   3.70915331,
         3.54056657,   3.37964236,   3.2260324 ,   3.07940424,
         2.93944057,   2.80583846,   2.67830877,   2.5565755 ])},
             scoring='neg_mean_absolute_error')

In [26]:
lasso_model = grid.best_estimator_
Yhat = lasso_model.predict(X_test)
np.fabs(Yhat - Y_test).mean()

257.2382010799497